
# Preprocesamiento

In [29]:
import numpy as np
import pandas as pd


In [30]:
# Se leen los datos del archivo csv
data = pd.read_csv("MovieGenre.csv", encoding ="ISO-8859-1")
data.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [31]:
print(data.shape)
data.isnull().sum()

(40108, 6)


imdbId          0
Imdb Link       0
Title           0
IMDB Score     48
Genre         145
Poster        725
dtype: int64

In [32]:
# Se remueven las peliculas que no cuentan con un género especificado.
data = data.dropna(subset=["Genre"])
data.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [33]:
data.isnull().sum()

imdbId          0
Imdb Link       0
Title           0
IMDB Score     46
Genre           0
Poster        700
dtype: int64

In [34]:
def divide_list(n_chunks, list_to_divide):
    chunk_size = len(list_to_divide)//n_chunks + 1
    chunks = [list_to_divide[i:i+chunk_size] for i in range(0, len(list_to_divide), chunk_size)]
    
    return chunks

n_chunks = 10
chunks = divide_list(n_chunks, data)
for chunk in chunks:
    print(len(chunk))
    
data = chunks[0]
data.head()

3997
3997
3997
3997
3997
3997
3997
3997
3997
3990


In [50]:
data.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,f
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...,0
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...,1
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,2
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,3
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...,4


In [61]:
new_data = {}
new_data['Poster'] = data['Poster']
new_data['IMDB Score'] = data['IMDB Score']


{}
{'alsdk': 0}


In [39]:
# Los datos faltantes del poster y score de las peliculas se obtendran por medio de un Scraping.
# Adicionalmente, se obtendrá el director, escritor y cast.
# Por último, se obtendrá el resumen y synopsis como data de texto.

import requests
from bs4 import BeautifulSoup
import csv
import time

links = data['Imdb Link']
page = requests.get(link)
soup = BeautifulSoup(page.content, "html.parser")

new_data = {}
new_data['Poster'] = data['Poster']
new_data['IMDB Score'] = data['IMDB Score']
new_data['Directors'] = []
new_data['Writers'] = []
new_data['Cast'] = []

for link in links:
    time.sleep(2)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    
    get_missing_info(new_data)
    get_text_info(new_data)    
    get_credits_info(new_data)
    
    
    new_data['Directors'].append(get_directors(soup))
    new_data['Writers'].append(get_writers(soup))
    new_data['Cast'].append(get_cast(soup))
    
    

    



0    http://www.imdb.com/title/tt114709
1    http://www.imdb.com/title/tt113497
2    http://www.imdb.com/title/tt113228
3    http://www.imdb.com/title/tt114885
4    http://www.imdb.com/title/tt113041
Name: Imdb Link, dtype: object

# WebScraping

In [19]:
# Los datos faltantes del género, poster y score de las peliculas se obtendran por medio de un Scraping.
# Adicionalmente, se obtendrá el director, escritor y cast.
# Por último, se obtendrá el resumen y synopsis como data de texto.

import requests
from bs4 import BeautifulSoup
import csv
import time

links = data['Imdb Link']

    
    
    
        
    with open(filename, "w") as csvfile:        
    field_names = ["Imdb Link", "Summary", "Synopsis", "Directors", ""]
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    writer.writeheader()

    for link in movie_links:
        time.sleep(2)
        csv_fields = {}
        csv_fields['Imdb Link'] = link
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            csv_fields['Summary'], csv_fields['Director'] = get_basic_info(soup)
            csv_fields['Synopsis Link'], csv_fields['Synopsis'] = get_synopsis(soup)                
            print("Ok en link: ", link)
        except Exception as ex:
            print(ex)
            print("Error en link: ", link)

        writer.writerow(csv_fields)



def get_synopsis(soup):    
    synopsis_link = None
    synopsis = None
    base_link = "http://www.imdb.com"
    
    storyline_block = soup.find(id="titleStoryLine")    
    if storyline_block is not None:
        see_more_block = storyline_block.find("span",{'class':"see-more inline"})
        sm_link_blocks = see_more_block.find_all("a")
        for sm_link_block in sm_link_blocks:
            if sm_link_block.get_text() == "Plot Synopsis":
                synopsis_link = sm_link_block['href']
                
                page = requests.get(base_link + synopsis_link)
                soup = BeautifulSoup(page.content, 'html.parser')
                synopsis = soup.find(id="plot-synopsis-content").get_text()
                break
                
    return synopsis_link, synopsis

def get_basic_info(soup):
    summary = soup.find("div", {'class': "summary_text"}).get_text()
    director = soup.find("span", {'itemprop': "director"}).get_text()
    #print(summary, director)
    return summary, director

def get_text_data(movie_links, filename):    
    #TODO
    #Add summary
    
    with open(filename, "w") as csvfile:        
        field_names = ["Imdb Link", "Summary", "Synopsis", "Directors", ""]
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        writer.writeheader()

        for link in movie_links:
            time.sleep(2)
            csv_fields = {}
            csv_fields['Imdb Link'] = link
            page = requests.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')
                        
            try:
                csv_fields['Summary'], csv_fields['Director'] = get_basic_info(soup)
                csv_fields['Synopsis Link'], csv_fields['Synopsis'] = get_synopsis(soup)                
                print("Ok en link: ", link)
            except Exception as ex:
                print(ex)
                print("Error en link: ", link)

            writer.writerow(csv_fields)
            
def divide_list(n_chunks, list_to_divide):
    chunk_size = len(list_to_divide)//n_chunks + 1
    chunks = [list_to_divide[i:i+chunk_size] for i in range(0, len(list_to_divide), chunk_size)]
    
    return chunks

#n_chunks = 10
#chunks = divide_list(n_chunks, data['Imdb Link'])
#for chunk in chunks:
#    print(len(chunk))

    
#import multiprocessing
#n_jobs = 1
#chunks = divide_list(n_jobs, data['Imdb Link'])
#
#jobs = []
#for i, chunk in enumerate(chunks):    
#    filename = "Movie_Synopsis_" + str(i) + ".csv"
#    p = multiprocessing.Process(target=get_text_data,
#                                args=(chunk, filename))
#    jobs.append(p)
#    p.start()
#    
#for job in jobs:
#    job.join()

4011
4011
4011
4011
4011
4011
4011
4011
4011
4009


In [20]:
i = 0
chunk_1 = get_text_data(chunks[i], "MovieSynopsis" + str(i) + ".csv")

Ok en link:  http://www.imdb.com/title/tt114709
Ok en link:  http://www.imdb.com/title/tt113497
Ok en link:  http://www.imdb.com/title/tt113228
Ok en link:  http://www.imdb.com/title/tt114885
Ok en link:  http://www.imdb.com/title/tt113041
Ok en link:  http://www.imdb.com/title/tt113277
Ok en link:  http://www.imdb.com/title/tt114319
Ok en link:  http://www.imdb.com/title/tt112302
Ok en link:  http://www.imdb.com/title/tt114576
Ok en link:  http://www.imdb.com/title/tt113189
Ok en link:  http://www.imdb.com/title/tt112346
Ok en link:  http://www.imdb.com/title/tt112896
Ok en link:  http://www.imdb.com/title/tt112453
Ok en link:  http://www.imdb.com/title/tt113987
Ok en link:  http://www.imdb.com/title/tt112760
Ok en link:  http://www.imdb.com/title/tt112641
Ok en link:  http://www.imdb.com/title/tt114388
Ok en link:  http://www.imdb.com/title/tt113101
Ok en link:  http://www.imdb.com/title/tt112281
Ok en link:  http://www.imdb.com/title/tt113845
Ok en link:  http://www.imdb.com/title/t

Ok en link:  http://www.imdb.com/title/tt113492
Ok en link:  http://www.imdb.com/title/tt113500
Ok en link:  http://www.imdb.com/title/tt113540
Ok en link:  http://www.imdb.com/title/tt113677
Ok en link:  http://www.imdb.com/title/tt113690
Ok en link:  http://www.imdb.com/title/tt107447
Ok en link:  http://www.imdb.com/title/tt113729
Ok en link:  http://www.imdb.com/title/tt113749
Ok en link:  http://www.imdb.com/title/tt113820
Ok en link:  http://www.imdb.com/title/tt113851
Ok en link:  http://www.imdb.com/title/tt110604
Ok en link:  http://www.imdb.com/title/tt110620
Ok en link:  http://www.imdb.com/title/tt113957
Ok en link:  http://www.imdb.com/title/tt113986
Ok en link:  http://www.imdb.com/title/tt114095
Ok en link:  http://www.imdb.com/title/tt114194
Ok en link:  http://www.imdb.com/title/tt114241
Ok en link:  http://www.imdb.com/title/tt114323
Ok en link:  http://www.imdb.com/title/tt114345
Ok en link:  http://www.imdb.com/title/tt114435
Ok en link:  http://www.imdb.com/title/t

Ok en link:  http://www.imdb.com/title/tt106339
Ok en link:  http://www.imdb.com/title/tt104779
Ok en link:  http://www.imdb.com/title/tt109348
Ok en link:  http://www.imdb.com/title/tt109444
Ok en link:  http://www.imdb.com/title/tt109446
Ok en link:  http://www.imdb.com/title/tt109484
Ok en link:  http://www.imdb.com/title/tt109504
Ok en link:  http://www.imdb.com/title/tt109506
Ok en link:  http://www.imdb.com/title/tt109450
Ok en link:  http://www.imdb.com/title/tt109813
Ok en link:  http://www.imdb.com/title/tt109830
Ok en link:  http://www.imdb.com/title/tt109831
Ok en link:  http://www.imdb.com/title/tt113305
Ok en link:  http://www.imdb.com/title/tt110091
Ok en link:  http://www.imdb.com/title/tt110093
Ok en link:  http://www.imdb.com/title/tt110167
Ok en link:  http://www.imdb.com/title/tt110213
Ok en link:  http://www.imdb.com/title/tt107472
Ok en link:  http://www.imdb.com/title/tt110357
Ok en link:  http://www.imdb.com/title/tt107426
Ok en link:  http://www.imdb.com/title/t

Ok en link:  http://www.imdb.com/title/tt107969
Ok en link:  http://www.imdb.com/title/tt111001
Ok en link:  http://www.imdb.com/title/tt107978
Ok en link:  http://www.imdb.com/title/tt107977
Ok en link:  http://www.imdb.com/title/tt107983
Ok en link:  http://www.imdb.com/title/tt105275
Ok en link:  http://www.imdb.com/title/tt108000
Ok en link:  http://www.imdb.com/title/tt108002
Ok en link:  http://www.imdb.com/title/tt108026
Ok en link:  http://www.imdb.com/title/tt105032
Ok en link:  http://www.imdb.com/title/tt108052
Ok en link:  http://www.imdb.com/title/tt111094
Ok en link:  http://www.imdb.com/title/tt108065
Ok en link:  http://www.imdb.com/title/tt111102
Ok en link:  http://www.imdb.com/title/tt108071
Ok en link:  http://www.imdb.com/title/tt111127
Ok en link:  http://www.imdb.com/title/tt111143
Ok en link:  http://www.imdb.com/title/tt108101
Ok en link:  http://www.imdb.com/title/tt108122
Ok en link:  http://www.imdb.com/title/tt111194
Ok en link:  http://www.imdb.com/title/t

Ok en link:  http://www.imdb.com/title/tt116289
Ok en link:  http://www.imdb.com/title/tt109593
Ok en link:  http://www.imdb.com/title/tt103095
Ok en link:  http://www.imdb.com/title/tt112368
Ok en link:  http://www.imdb.com/title/tt104046
Ok en link:  http://www.imdb.com/title/tt62952
Ok en link:  http://www.imdb.com/title/tt115742
Ok en link:  http://www.imdb.com/title/tt117420
Ok en link:  http://www.imdb.com/title/tt115951
Ok en link:  http://www.imdb.com/title/tt117784
Ok en link:  http://www.imdb.com/title/tt117107
Ok en link:  http://www.imdb.com/title/tt117979
Ok en link:  http://www.imdb.com/title/tt95776
Ok en link:  http://www.imdb.com/title/tt115851
Ok en link:  http://www.imdb.com/title/tt116322
Ok en link:  http://www.imdb.com/title/tt109374
Ok en link:  http://www.imdb.com/title/tt110712
Ok en link:  http://www.imdb.com/title/tt112817
Ok en link:  http://www.imdb.com/title/tt113362
Ok en link:  http://www.imdb.com/title/tt73778
Ok en link:  http://www.imdb.com/title/tt11

...

Ok en link:  http://www.imdb.com/title/tt78902
Ok en link:  http://www.imdb.com/title/tt91217
Ok en link:  http://www.imdb.com/title/tt94812
Ok en link:  http://www.imdb.com/title/tt72890
Ok en link:  http://www.imdb.com/title/tt69704
Ok en link:  http://www.imdb.com/title/tt42208
Ok en link:  http://www.imdb.com/title/tt49730
Ok en link:  http://www.imdb.com/title/tt62886
Ok en link:  http://www.imdb.com/title/tt51411
Ok en link:  http://www.imdb.com/title/tt57344
Ok en link:  http://www.imdb.com/title/tt94731
Ok en link:  http://www.imdb.com/title/tt74235
Ok en link:  http://www.imdb.com/title/tt64110
Ok en link:  http://www.imdb.com/title/tt68323
Ok en link:  http://www.imdb.com/title/tt104057
Ok en link:  http://www.imdb.com/title/tt42393
Ok en link:  http://www.imdb.com/title/tt41705
Ok en link:  http://www.imdb.com/title/tt35966
Ok en link:  http://www.imdb.com/title/tt118043
Ok en link:  http://www.imdb.com/title/tt53137
Ok en link:  http://www.imdb.com/title/tt39748
Ok en link:

Ok en link:  http://www.imdb.com/title/tt149367
Ok en link:  http://www.imdb.com/title/tt166175
Ok en link:  http://www.imdb.com/title/tt236216
Ok en link:  http://www.imdb.com/title/tt160644
Ok en link:  http://www.imdb.com/title/tt179063
Ok en link:  http://www.imdb.com/title/tt64180
Ok en link:  http://www.imdb.com/title/tt83833
Ok en link:  http://www.imdb.com/title/tt102898
Ok en link:  http://www.imdb.com/title/tt68327
Ok en link:  http://www.imdb.com/title/tt56687
Ok en link:  http://www.imdb.com/title/tt93776
Ok en link:  http://www.imdb.com/title/tt51383
Ok en link:  http://www.imdb.com/title/tt48140
Ok en link:  http://www.imdb.com/title/tt85701
Ok en link:  http://www.imdb.com/title/tt74860
Ok en link:  http://www.imdb.com/title/tt80487
Ok en link:  http://www.imdb.com/title/tt176783
Ok en link:  http://www.imdb.com/title/tt199725
Ok en link:  http://www.imdb.com/title/tt141926
Ok en link:  http://www.imdb.com/title/tt159097
Ok en link:  http://www.imdb.com/title/tt104549
Ok

Ok en link:  http://www.imdb.com/title/tt105316
Ok en link:  http://www.imdb.com/title/tt102951
Ok en link:  http://www.imdb.com/title/tt100046
Ok en link:  http://www.imdb.com/title/tt94882
Ok en link:  http://www.imdb.com/title/tt92710
Ok en link:  http://www.imdb.com/title/tt93011
Ok en link:  http://www.imdb.com/title/tt187078
Ok en link:  http://www.imdb.com/title/tt179074
Ok en link:  http://www.imdb.com/title/tt182295
Ok en link:  http://www.imdb.com/title/tt145503
Ok en link:  http://www.imdb.com/title/tt162711
Ok en link:  http://www.imdb.com/title/tt143422
Ok en link:  http://www.imdb.com/title/tt99726
Ok en link:  http://www.imdb.com/title/tt77362
Ok en link:  http://www.imdb.com/title/tt82009
Ok en link:  http://www.imdb.com/title/tt74156
Ok en link:  http://www.imdb.com/title/tt93605
Ok en link:  http://www.imdb.com/title/tt102592
Ok en link:  http://www.imdb.com/title/tt67741
Ok en link:  http://www.imdb.com/title/tt71360
Ok en link:  http://www.imdb.com/title/tt82220
Ok 

Ok en link:  http://www.imdb.com/title/tt234137
Ok en link:  http://www.imdb.com/title/tt161216
Ok en link:  http://www.imdb.com/title/tt157182
Ok en link:  http://www.imdb.com/title/tt209322
Ok en link:  http://www.imdb.com/title/tt144964
Ok en link:  http://www.imdb.com/title/tt259207
Ok en link:  http://www.imdb.com/title/tt216772
Ok en link:  http://www.imdb.com/title/tt187696
Ok en link:  http://www.imdb.com/title/tt171580
Ok en link:  http://www.imdb.com/title/tt190798
Ok en link:  http://www.imdb.com/title/tt204626
Ok en link:  http://www.imdb.com/title/tt202677
Ok en link:  http://www.imdb.com/title/tt181875
Ok en link:  http://www.imdb.com/title/tt211938
Ok en link:  http://www.imdb.com/title/tt191037
Ok en link:  http://www.imdb.com/title/tt197384
Ok en link:  http://www.imdb.com/title/tt134630
Ok en link:  http://www.imdb.com/title/tt210717
Ok en link:  http://www.imdb.com/title/tt182508
Ok en link:  http://www.imdb.com/title/tt188694
Ok en link:  http://www.imdb.com/title/t

Ok en link:  http://www.imdb.com/title/tt212423
Ok en link:  http://www.imdb.com/title/tt213446
Ok en link:  http://www.imdb.com/title/tt104765
Ok en link:  http://www.imdb.com/title/tt102388
Ok en link:  http://www.imdb.com/title/tt95690
Ok en link:  http://www.imdb.com/title/tt105165
Ok en link:  http://www.imdb.com/title/tt69897
Ok en link:  http://www.imdb.com/title/tt71517
Ok en link:  http://www.imdb.com/title/tt95348
Ok en link:  http://www.imdb.com/title/tt108451
Ok en link:  http://www.imdb.com/title/tt186589
Ok en link:  http://www.imdb.com/title/tt209475
Ok en link:  http://www.imdb.com/title/tt118614
Ok en link:  http://www.imdb.com/title/tt116119
Ok en link:  http://www.imdb.com/title/tt157122
Ok en link:  http://www.imdb.com/title/tt178642
Ok en link:  http://www.imdb.com/title/tt234805
Ok en link:  http://www.imdb.com/title/tt188506
Ok en link:  http://www.imdb.com/title/tt162024
Ok en link:  http://www.imdb.com/title/tt216800
Ok en link:  http://www.imdb.com/title/tt925

In [21]:
data = pd.read_csv("MovieSynopsis0.csv")

In [22]:
data.head()

,Imdb Link,Synopsis Link,Synopsis,Summary,Director
0,http://www.imdb.com/title/tt114709,/title/tt0114709/synopsis?ref_=tt_stry_pl,\nA boy called Andy Davis (voice: John Morris)...,\n A cowboy doll is profoun...,\nJohn Lasseter
1,http://www.imdb.com/title/tt113497,/title/tt0113497/synopsis?ref_=tt_stry_pl,\nThe film begins in 1869 in the town of Brant...,\n When two kids find and p...,\nJoe Johnston
2,http://www.imdb.com/title/tt113228,NaN,NaN,\n John and Max resolve to ...,\nHoward Deutch
3,http://www.imdb.com/title/tt114885,NaN,NaN,\n Based on Terry McMillan'...,\nForest Whitaker
4,http://www.imdb.com/title/tt113041,NaN,NaN,\n George Banks must deal n...,\nCharles Shyer


In [24]:
data.isnull().sum()

Imdb Link           0
Synopsis Link    2244
Synopsis         2244
Summary             4
Director            4
dtype: int64